In [1]:
import pandas as pd

In [2]:
import tweepy,twitter

In [74]:
auth = tweepy.OAuthHandler('D4cA5heqwzCJryH1GnUzEDopn', 'wjSCGp8t4uP3M1egnzy8lhmja3JR04Pw2B2D01gZHz0Kf6iSvt')
auth.set_access_token('823928942084005889-LCht7E7SJkIya0CBWtiCrbepRrG1KuU', 'wIllRVmp2X2qEra4UYiWGYG5wpWVZo05acQvJ6sf28Iz0')

api = tweepy.API(auth)

In [5]:
suicidal_tweets=pd.read_csv("suicidal_data.csv")

In [6]:
import pickle
import re
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor


In [7]:
def preprocess_tweet(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text+' '.join(emoticons).replace('-', '') 
    return text

In [8]:
tqdm.pandas()
suicidal_tweets['tweet'] = suicidal_tweets['tweet'].progress_apply(preprocess_tweet)

C:\Users\user\Desktop\sample_project\env\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████| 9119/9119 [00:00<00:00, 13524.01it/s]


In [9]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [11]:
[w for w in tokenizer_porter('a swimmer likes swimming and swims a lot') if w not in stop]

['swimmer', 'like', 'swim', 'swim', 'lot']

In [12]:

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text += ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in tokenizer_porter(text) if w not in stop]
    return tokenized

## Using the Hashing Vectorizer

In [13]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, 
                         preprocessor=None,tokenizer=tokenizer)

## Building the ml model

In [14]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1)

In [15]:
X = suicidal_tweets["tweet"].to_list()
y = suicidal_tweets['label']

## Training the model

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [48]:
classes = np.array([0, 1])
clf.partial_fit(X_train, y_train,classes=classes)

SGDClassifier(loss='log', random_state=1)

In [49]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.912


In [50]:
clf = clf.partial_fit(X_test, y_test)

## Testing 

In [51]:
label = {0:'negative', 1:'positive'}

In [52]:
ideation = ["suicide","suicidal", "kill myself", "my suicide note", "my suicide letter", "end my life", "never wake up", "can't go on"," cannot go on", "not worth living", "ready to jump", "swleep forever", "want to die", "be dead", "better off without me", "better off dead", "don't want to being here", "tired of living", "die alone", "go to sleep forever", "wanna die", "wanna suicide", "commit suicide", "slit my wrist","cut my wrist", "slash my wrist", "do not want to be here", "want it to be over", "want to be dead", "nothing to live for", "not worth living","ready to die", "thoughts of suicide", "thoughts of killing myself", "why should i live", "take my own life", "depressed"]

In [242]:
streamed_tweets=[]
for query in ideation:    
    tweets = api.search(q= query + " -filter:retweets", lang="en",show_user=True, result_type="recent")
    for i in tweets:
        print(i.text)
        streamed_tweets.append({'tweet_id': i.id,'tweet_text': i.text, 'user_name': i.user.screen_name})

"There really is no place for me in a world like this!", states @SyHawkes.

Firstly Sy, stop playing the suicide ca… https://t.co/RqbNUffsPQ
@Sp_rig @PriestlyGhost suicide
The Emmys are committing suicide by giving an award to Andrew Cuomo.

I mean, think about it, the Emmys are more th… https://t.co/ESLXRwC0sZ
@realDonaldTrump donald trump suicide watch before it disappears
💫💫💫💫💫💫💫💫💫💫💫💫
https://t.co/9YYEm62tI0
@DeadByBHVR To my noob friends I don't tell them there is a 4% unhook chance, I tell em it's the suicide button
@ShannonSharpe Skip on suicide watch rn
@zeinabfarhoudje i would bully me to the point of suicide if i didnt know me personally
@LaurenWitzkeDE @BeComfy @GovKemp @pdabrosca I’ve just seen for the first time the j immigration plan,man oh man,th… https://t.co/IYbF8jIQtO
@LolaMinnesota @cheryldgb @GovernorVA @FEA_Fairfax Obviously, a lot more help. 

It's like when parents are dealing… https://t.co/kQi9bxJ7Zt
@donwinslow @LeaBlackMiami White people have signed to voluntar

In [243]:
streamed_tweet_df=pd.DataFrame.from_dict(streamed_tweets)
streamed_tweet_df

,tweet_id,tweet_text,user_name
0,1330274464488108042,"""There really is no place for me in a world li...",The_Unredacted
1,1330274440169533444,@Sp_rig @PriestlyGhost suicide,muffprettyok
2,1330274413875507210,The Emmys are committing suicide by giving an ...,RightyTeacher
3,1330274409735725059,@realDonaldTrump donald trump suicide watch be...,Aleynatilki_off
4,1330274402789896195,@DeadByBHVR To my noob friends I don't tell th...,gagelongdon
...,...,...,...
523,1330274512055791618,Why lil baby have to make emotionally scarred ...,kickbruhtowski
524,1330274497182769154,i lost 5lbs &amp; im feeling a bit less depres...,chubbiiibunniii
525,1330274458871934976,Charlie Kaufman changed my life because now I'...,dexterfilms_
526,1330274441130029063,Zetsu keeps playing sad trumpet sounds each ti...,ParodyObito


In [244]:
analysis= pd.DataFrame(columns={"Prediction","Probability"})
analysis["Prediction"].astype(object)
analysis["Probability"].astype(float)

Series([], Name: Probability, dtype: float64)

In [245]:
for row in range(0,len(streamed_tweet_df)):
    example=[streamed_tweet_df.iloc[row,1]]
    X = vect.transform(example)
    print('Prediction: %s   Probability: %.2f%%'
    %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))
    analysis=analysis.append({'Prediction': label[clf.predict(X)[0]],
                                             'Probability': np.max(clf.predict_proba(X))*100},
                                            ignore_index=True)

Prediction: positive   Probability: 72.45%
Prediction: positive   Probability: 73.56%
Prediction: negative   Probability: 60.98%
Prediction: negative   Probability: 77.19%
Prediction: positive   Probability: 78.45%
Prediction: negative   Probability: 77.81%
Prediction: positive   Probability: 87.71%
Prediction: negative   Probability: 94.73%
Prediction: negative   Probability: 78.83%
Prediction: negative   Probability: 67.70%
Prediction: negative   Probability: 77.19%
Prediction: negative   Probability: 65.16%
Prediction: negative   Probability: 77.24%
Prediction: negative   Probability: 96.29%
Prediction: negative   Probability: 69.97%
Prediction: negative   Probability: 94.19%
Prediction: negative   Probability: 95.88%
Prediction: negative   Probability: 59.14%
Prediction: positive   Probability: 98.34%
Prediction: positive   Probability: 67.50%
Prediction: positive   Probability: 97.57%
Prediction: positive   Probability: 51.46%
Prediction: negative   Probability: 78.66%
Prediction:

In [246]:
analysis

,Probability,Prediction
0,72.454165,positive
1,73.559244,positive
2,60.981791,negative
3,77.186870,negative
4,78.447839,positive
...,...,...
523,59.764615,positive
524,85.958842,negative
525,58.576167,negative
526,62.094080,negative


In [247]:
mask = (analysis['Probability'].ge(80)) & (analysis['Prediction']=='positive')
analysis['mask'] = np.where(mask,'Flagged', 'no problem')
print(analysis), analysis['mask'].dtypes


     Probability Prediction        mask
0      72.454165   positive  no problem
1      73.559244   positive  no problem
2      60.981791   negative  no problem
3      77.186870   negative  no problem
4      78.447839   positive  no problem
..           ...        ...         ...
523    59.764615   positive  no problem
524    85.958842   negative  no problem
525    58.576167   negative  no problem
526    62.094080   negative  no problem
527    59.847791   negative  no problem

[528 rows x 3 columns]


(None, dtype('O'))

In [253]:
for row in range(len(analysis)):
    if analysis['mask'][row]=='Flagged':
        users=streamed_tweets[row]
        print(row,users['user_name'],users['tweet_text'])

6 blinczyk @zeinabfarhoudje i would bully me to the point of suicide if i didnt know me personally
18 fyzqwx still wandering, how it's feels to live a life without suicidal thoughts.
20 kdd0116 How did I go from being suicidal to this fuck me
28 splatterism @iamvegetabIe And now we’re suicidal
45 F8webb Every time an assignment gets posted my suicide note does get longer
48 JbShibe @prodegyxtinct @gawrgura I'm mentioning this in my suicide note
52 Dyingathreeam btw this is my suicide note
57 celestiansoul Tw // suicide 

Last night, my internet friend sent me a suicide note. Now, I will never hear from her again. I’ll… https://t.co/eKefec2Fft
61 dongjunstan tw // suicide

they think that i like being here when she saw my fucking suicide letter
68 diptychbitch in the first suicide letter I ever wrote when I was 8 years old I list one of my reasons as the desire to meet Theo… https://t.co/SSOr5qzF4y
244 lexlovesnaps I am so tired of living this pointless life
302 TXMLINSQN tw // suicide 